In [42]:
import pandas as pd
import sweetviz as sv
import numpy as np
import os

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

In [2]:
data_path = os.path.join('..', 'data', 'SkuChangeRequests.csv')

In [3]:
columns_ru = ['Номер заявки', 'Форма заявки', 'Наименование', 'Единица измерения', 'Изображение', 'Классификация ГОСТ/ТУ', 'Описание', 'Статус', 'Причина отказа', 'Комментарий', 'Ид оферты', 'Наименование оферты', 'Артикул оферты', 'Регион поставки', 'Срок поставки в днях От', 'Срок поставки в днях До', 'Доступное количество От', 'Доступное количество До', 'Исходные характеристики', 'Категория оферты', 'Категория справочника', 'Вид продукции', 'Количество эталонных утвержденных характеристик в категории', 'Количество использованных поставщиком эталонных утвержденных характеристик']
columns_en = ['application_num', 'application_form', 'name', 'measurement_unit', 'image', 'gost_calss', 'description', 'status', 'reason', 'comment' , 'offer_id', 'offer_name', 'offer_number', 'delivery_region', 'delivery_from','delivery_to', 'quantity_from', 'quantity_to', 'init_char', 'offer_category', 'directory_category', 'product_type', 'number_characteristics', 'number_characteristics_supplier']
columns_translator = {ru: eng for ru, eng in zip(columns_ru, columns_en)}

In [4]:
df = pd.read_csv(data_path, sep=',')

In [5]:
df = df.rename(columns=columns_translator)

In [18]:
def sep_reason(reason: str):
    if pd.notna(reason):
        return reason.split(';')
    return reason

In [76]:
reasons_df = df.reason.dropna().apply(sep_reason)

reas = [item for sublist in reasons_df.to_list() for item in sublist]
reas = pd.DataFrame(reas, columns=['reason'])

reject_reasons = {rej: i for i, rej in  enumerate(reas.reason.value_counts().index.to_list())}

In [77]:
REASON_NUM = 55

def get_reject_reason_vector_index(reason):
    if isinstance(reason, list):
        return [reject_reasons.get(reas, 0) for reas in reason]
    return []
    
def get_reject_reason_vector(reason_index):
    vec = np.zeros(REASON_NUM, dtype=int)
    vec[reason_index] = 1
    return vec
            

In [78]:
df.reason.apply(sep_reason).apply(get_reject_reason_vector_index).apply(get_reject_reason_vector)

0         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1         [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...
2         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
3         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                ...                        
199995    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
199996    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
199997    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
199998    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
199999    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: reason, Length: 200000, dtype: object

In [73]:
df.reason.dropna().apply(sep_reason).apply(len).value_counts()

1     48586
2     18928
3     13172
4      7021
5      3362
6      1519
7       639
8       251
9       114
10       42
11        8
12        4
13        1
Name: reason, dtype: int64

In [13]:
my_report = sv.analyze(df)
my_report.show_html('report.html')

                                             |          | [  0%]   00:00 -> (? left)

Report report.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.
